<a href="https://colab.research.google.com/github/OnlyBelter/machine-learning-note/blob/master/pytorch/pytorch_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:


*   https://pytorch.org/tutorials/beginner/basics/intro.html
*   https://youtu.be/OMDn66kM9Qc?si=p_CggEi2Rvhv6NKK



In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
device = (
"cuda"
if torch.cuda.is_available()
else "mps"
if torch.backends.mps.is_available()
else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
learning_rate = 0.001
batch_size = 64
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size=64):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        ## Compute prediction and loss

        # 1 forward
        pred = model(X)

        # 2 compute the objective function
        loss = loss_fn(pred, y)

        # 3 cleaning the gradients
        optimizer.zero_grad()

        ## Backpropagation
        # 4 accumulate the partial derivatives of loss wrt params
        loss.backward() # params.grad._sum(dloss/dparams)

        # 5 step in the opposite direction of the gradient
        optimizer.step() # with torch.no_grad(): params = params - eta * params.grad

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.5MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 229kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 4.32MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 15.8MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.293028  [   64/60000]
loss: 2.281813  [ 6464/60000]
loss: 2.266456  [12864/60000]
loss: 2.261850  [19264/60000]
loss: 2.253941  [25664/60000]
loss: 2.225630  [32064/60000]
loss: 2.231944  [38464/60000]
loss: 2.198348  [44864/60000]
loss: 2.190732  [51264/60000]
loss: 2.170475  [57664/60000]
Test Error: 
 Accuracy: 52.1%, Avg loss: 2.157943 

Epoch 2
-------------------------------
loss: 2.166313  [   64/60000]
loss: 2.149914  [ 6464/60000]
loss: 2.104220  [12864/60000]
loss: 2.119308  [19264/60000]
loss: 2.063360  [25664/60000]
loss: 2.013458  [32064/60000]
loss: 2.033087  [38464/60000]
loss: 1.955152  [44864/60000]
loss: 1.963674  [51264/60000]
loss: 1.889008  [57664/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.889048 

Epoch 3
-------------------------------
loss: 1.918791  [   64/60000]
loss: 1.883032  [ 6464/60000]
loss: 1.781850  [12864/60000]
loss: 1.818143  [19264/60000]
loss: 1.699528  [25664/60000]
loss: 1.663056  [32064/600